<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一、引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二、工具调用代码示例" data-toc-modified-id="二、工具调用代码示例">二、工具调用代码示例</a></span>
         <ul class="toc-item">
             <li><span><a href="##2.1 从 Python 函数定义工具接口" data-toc-modified-id="2.1 从 Python 函数定义工具接口">2.1 从 Python 函数定义工具接口</a></span></li>
             <li><span><a href="##2.2 定义自动检索工具" data-toc-modified-id="2.2 定义自动检索工具">2.2 定义自动检索工具</a></span></li>
             <li><span><a href="##2.3 综合的工具选择系统" data-toc-modified-id="2.3 综合的工具选择系统">2.3 综合的工具选择系统</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三、本章小结" data-toc-modified-id="三、本章小结">三、本章小结</a></span></li>
     </ul>
</div>

## 一、引言
***

<div style="display:none;">大语言模型 (Large Language Models, LLMs) 的优点之一是能够与外部环境交互并采取行动。为了实现这一目标，LLMs 需要使用有效且可靠的接口，我们称之为<strong>工具调用 (Tool Calling)</strong>。</div>

在基础的检索增强生成 (Retrieval-Augmented Generation, RAG) 管道中，大语言模型 (Large Language Models, LLMs)  仅用于生成结果。在第二章我们已经展示了如何以略微更复杂的方式使用 LLMs，即利用它来选择最佳的查询引擎来回答用户的查询。这是<strong>工具调用 (Tool Calling)</strong> 的简化形式。

<div style="display:none;">大在这个信息爆炸的时代，我们面临着一个前所未有的挑战：如何从海量数据中快速、准确地检索和处理信息。本教程正是为了应对这一挑战而设计的，它将引导我们进入一个由大语言模型（Large Language Models，LLMs）和工具调用（Tool Calling）技术共同构建的智能信息处理新世界。</div>

在本章中，LLMs 将不仅被用于选择最佳的查询引擎，还推断出要传递给函数的参数。这使得 LLMs 能够利用向量数据库，而不仅仅是使用其输出。通过工具调用，用户可以进行更多元化的查询，并且获得比标准RAG技术更精确的结果。

<div style="display:none;">在本教程中，我们将探索一种革命性的方法，这种方法不仅能够让 LLMs 生成文本，还能使它们在决策过程中扮演更为关键的角色。想象一下，**一个系统能够理解您的查询，自动选择最合适的工具来处理问题，并准确地推断出需要传递给这些工具的参数**。这不再是科幻小说中的场景，而是我们即将揭开的技术现实。

我们将从 RAG 管道的基础原理出发，逐步深入到更高级的工具调用概念。您将学习到如何定义工具接口，并通过 Python 函数的签名自动推断参数，这是 LlamaIndex 索引抽象的核心。我们将通过实际的代码示例，展示如何将这些工具传递给 LLM，并利用 predict_and_call 函数来实现智能决策和执行。

教程的重点之一是展示 LLMs 如何与向量数据库交互，通过元数据过滤器来优化搜索结果，从而提供更精确的信息检索。这不仅提高了效率，还极大地增强了用户体验。随着代码示例的深入，我们将构建一个复杂的智能体层，它将展示 LLMs 在选择工具和推断参数方面的高级能力。最终，我们将整合所有这些技术，构建一个完整的工具选择系统，它能够理解复杂的用户查询，并提供精确、有用的答案。结果是用户可以提出更多问题，并**通过工具调用获得比标准 RAG 技术更精确的结果**。

本教程不仅适合那些对大语言模型和 LLM-based Agent 感兴趣的学生和专业人士，也适合任何希望在信息检索和处理领域保持领先地位的人。让我们一起开启这段激动人心的学习之旅，探索如何利用 LLMs 和工具调用技术，构建下一代 Agentic RAG 系统。那么，让我们开始编码吧~ </div>


In [16]:
from dotenv import load_dotenv, find_dotenv

# 你需要在文件目录下创建一个.env的文件，文件中存放你的 OPENAI_API_KEY=“your OPENAI API Key”
# 从 .env 文件将 OPENAI_API_KEY 加载为环境变量。
# 当后续使用到 OPENAI 模型时，环境变量会直接被使用于认证(Authentication)。
_ = load_dotenv(find_dotenv())

In [17]:
import nest_asyncio

nest_asyncio.apply()

<br>
LlamaIndex 为了实现高效加载和处理数据，有些功能使用了asyncio实现异步编程优化 I/O 操作。然而，asyncio 的设计本身不支持事件循环的嵌套使用，这意味着在已经运行事件循环的环境中，尝试再次启动事件循环或同步运行任务会导致异常，通常会抛出 "RuntimeError: This event loop is already running" 错误。 在Jupyter Notebook这样的交互式环境中，因为每个代码单元格（cell）都可能尝试创建或操作事件循环。从而出现事件循环的嵌套使用。

为了解决这个问题，nest_asyncio 模块对 asyncio 进行了补丁，允许嵌套使用 asyncio 的函数，例如 asyncio.run() 和 loop.run_until_complete()。这种补丁修改了 asyncio 的行为，临时允许 asyncio 在已有事件循环的环境中正常工作，确保在 Jupyter Notebook 环境中，即使存在已运行的事件循环，也能够正确执行异步任务。

<br>

## 二、 简单的工具调用

工具调用分为以下几个步骤：
- 构建Python函数 (Python function)
- 为给定的函数定义工具接口（Tool Interface）
- LLM 使用 LlamaIndex 抽象（LlamaIndex abstractions）根据函数的签名（Signature）来推断参数

让我们来看一个简单的工具调用例子，了解工具调用的工作原理。

### 2.1、定义Python函数

首先，我们定义一个 add 函数和一个 mystery 函数。 值得注意的是，在定义函数时，我们提供了文档字符串、参数列表及其类型用于描述函数的输入输出。这些信息加上函数名本身为函数签名（Signature）。这不仅仅是为了编程规范目的，它后续会被用于生成 LLM 的提示（Prompts）。


In [18]:
def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

<br>

### 2.2、定义工具接口

LlamaIndex 中的核心抽象是函数工具（**FunctionTool**）。**此函数工具会包装您提供给它的任何具体的 Python 函数**，所以，我们看到函数工具接受了这里定义的 add 函数以及一个 mystery 函数（也就是 x + y 乘以 x + y）。您可以看到 add 和 mystery 都有 x 和 y 变量的**类型注释**，还有文档字符串（**docstring**）。这不仅仅是为了编程规范目的，这实际上很重要，因为这些东西将被用作 **LLM 的提示（Prompts）**。目前，LlamaIndex 的函数工具已经与许多 LLM 提供商（包括 OpenAI）的**函数调用功能**进行了原生集成。

<br>

In [36]:
from llama_index.core.tools import FunctionTool

add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [40]:
add_tool.metadata.to_openai_function()

/var/folders/6c/tn_x_scj0klffm7j_pj2mf7c0000gn/T/ipykernel_73012/307452021.py:1: DeprecationWarning: Call to deprecated method to_openai_function. (Deprecated in favor of `to_openai_tool`, which should be used instead.)
  add_tool.metadata.to_openai_function()


{'name': 'add',
 'description': 'add(x: int, y: int) -> int\nAdds two integers together.',
 'parameters': {'type': 'object',
  'properties': {'x': {'title': 'X', 'type': 'integer'},
   'y': {'title': 'Y', 'type': 'integer'}},
  'required': ['x', 'y']}}

In [41]:
# import inspect
# print(inspect.getsource(FunctionTool))
# from inspect import signature

### 2.2、使用 LLMs 进行工具调用


接下来，我们要将定义好的工具`add_tool` 和 `mystery_tool` 传递给大语言模型（LLM）。首先，加载大语言模型，这里选择的模型为**gpt-3.5-turbo**。 然后我们调用 `predict_and_call` 函数。该函数的输入为：

- 工具列表(tools)： 一组工具
- 用户查询输入(user_msg)： 输入提示字符串或一系列聊天消息

然后它既能够决定要调用的工具，也能够调用工具本身，并返回最终的响应。

初始化大语言模型，这里悬着

In [19]:
from llama_index.llms.openai import OpenAI

# 加载大语言模型
llm = OpenAI(model="gpt-3.5-turbo")

# 调用 predict_and_call 函数
response = llm.predict_and_call(
    tools=[add_tool, mystery_tool],
    user_msg="Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121


In [20]:
response

AgentChatResponse(response='121', sources=[ToolOutput(content='121', tool_name='mystery', raw_input={'args': (), 'kwargs': {'x': 2, 'y': 9}}, raw_output=121, is_error=False)], source_nodes=[], is_dummy_stream=False)

这里我们看到了调用函数 mystery 的中间步骤。函数传入参数：x 等于 2，y 等于 9。所以我们看到，代码实现调用了正确的工具，也推断出了正确的参数，输出是 121，11 乘以 11 是 121，所以我们得到了正确的答案。但请注意，这个简单的示例实际上是路由（router）的扩展版本。**LLM 不仅选择合适的工具，还要决定给工具给予什么参数**。

下面的 2.2 定义自动检索工具小节，让我们使用这个关键概念在向量搜索之上定义一个稍微复杂一点的智能体层。**LLM 不仅可以选择向量搜索，我们还可以让它推断元数据过滤器**，这是一个结构化的标签列表，有助于返回一组更精确的搜索结果。我们将使用与前面相同的论文 MetaGPT，这次让我们关注节点本身，或者块，因为我们将查看附加到这些块的实际元数据。


### 2.3、中文示例

In [5]:
def add(x: int, y: int) -> int:
    """将两个整数相加。"""
    return x + y


def mystery(x: int, y: int) -> int:
    """在两个数字之上运行的神秘函数。"""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    tools=[add_tool, mystery_tool], 
    user_msg="告诉我 mystery 函数对 2 与 9 输出的结果", 
    verbose=True
)

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121 <class 'llama_index.core.chat_engine.types.AgentChatResponse'>


## 三、 定义自动检索工具

这里需要用到的文档为`metagpt.pdf`。已经在当前目录下面。 或者你也可以通过下面的指令来下载文件。

```bash
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf
```

`wget` 为linux系统下的下载指令。如果你的系统为Windows或者Mac, 则可以通过curl来下载。

```bash
!curl -s -o metagpt.pdf "https://openreview.net/pdf?id=VtmBAGCN7o"
```

### 3.1、加载数据为文档格式

使用 LlamaIndex 中的简单目录阅读器来加载此 pdf 文件

In [22]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["data/metagpt.pdf"]).load_data()

### 3.2、切割文档为节点格式

使用句子分割器将这些文档分割成一组节点，每个节点的Token个数少于1024个。

In [23]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

<br>

这里的每个节点都代表一个块(chunk)，我们可以通过执行 `node.getContent` 来看看节点的内容。同时可以将元数据模式(metadata_mode)设置为 `all`，不仅可以打印节点本身的文本内容，还可以打印附加到文档的元数据。

<br>

In [24]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-15
last_modified_date: 2024-06-08

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, howe

<br>

上面是第一个节点的打印结果。我们不仅得到了论文文本，顶部还包含了文本的元数据(Meta Data)：

- 页标签（page_label）: 1
- 文件名（file_name）: metagpt.pdf
- 文件路径（file_path）: data/metagpt.pdf
- 文件类型（file_type）: application/pdf
- 文件大小（file_size）: 16911937
- 创建日期（creation_date）: 2024-06-15
- 上次修改日期（last_modified_date）: 2024-06-08

<br>

### 3.3、定义向量存储索引

接下来，我们将在这些节点上定义一个**向量存储索引**

In [26]:
from llama_index.core import VectorStoreIndex

# 向量存储索引
vector_index = VectorStoreIndex(nodes)

### 3.4、创建查询引擎

In [27]:
# 基于向量索引vector_index构建向量查询引擎
query_engine = vector_index.as_query_engine(similarity_top_k=2)

这船

我们实际上可以尝试通过元数据过滤器查询此 RAG 管道，只是为了向您展示元数据过滤的工作原理。因此，我们导入这个名为 `MetadataFilters` 的对象，然后我们只需指定一个过滤器，其中页面标签的值设置 "2"；另外，similarity_top_k 也等于 2。

因此，我们将其定义为查询引擎，并且我们查询 MetaGPT 论文中的一些高水平结果。如果我们观察一下源节点，一旦我们运行它，我们查询到 MetaGPT 的一些高水平结果。当我们收到响应时，我们首先看一下响应字符串，它概述了 MetaGPT 的总体结果。但至关重要的是，我们将查看页码。我们迭代源节点时，实际上可以打印出**附加到这些源节点的元数据**。我们看到这里的页面标签等于 "2"。因此我们观察到它能够正确过滤页码，**仅将搜索限制为页面标签等于 2 的页面**。这说明我们的查询引擎很聪明，它知道我们只想看标签为 "2" 的论文页面的信息。这样，我们就不会被其他不相关的页面干扰，可以更快地找到我们想要的答案。


In [11]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts([{"key": "page_label", "value": "2"}]),  # 根据页码过滤
)

### 3.5、进行查询

In [12]:
response = query_engine.query("What are some high-level results of MetaGPT?",)
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with 85.9% and 87.7% in Pass@1, outperforming other popular frameworks like AutoGPT, LangChain, AgentVerse, and ChatDev. Additionally, MetaGPT excels in handling higher levels of software complexity and providing extensive functionality, as evidenced by achieving a 100% task completion rate in experimental evaluations.


In [13]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


In [14]:
response = query_engine.query( "MetaGPT 取得了哪些高水平的结果？")
print(str(response))

In [15]:
print(str(response))
print("-" * 88)
for n in response.source_nodes:
    print(n.metadata)

MetaGPT在代码生成基准测试中取得了85.9%和87.7%的一级通过率，超越了其他流行的框架，如AutoGPT，LangChain，AgentVerse和ChatDev。此外，在处理更高级别的软件复杂性和提供广泛功能方面，MetaGPT也表现出色。
----------------------------------------------------------------------------------------
{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


### 3.6、封装函数

最后我们将前面的部分构建成一个函数`vector_query`

本教程的最后一节将这个整体检索工具包装成一个函数。这个函数接受查询字符串和页码作为过滤器。**LLM 实际上可以推断要过滤用户查询的页码，而不是让用户手动指定元数据过滤器**。

这里需要注意的是，元数据其实不仅限于页码。**您可以通过 LlamaIndex 抽象定义任何想要的元数据**，如节 IDs、页眉、页脚或其他任何内容。使用许多元数据过滤器的能力在 GPT-4 等更好的大语言模型中尤为突出，因此我们强烈建议读者自行实践一下。

这里我们将定义一个封装它的 Python 函数。我们定义一个名为 **vector_query** 的函数，它接受查询和页码。这允许您对索引执行向量搜索，并指定页码作为元数据过滤器。最后，我们定义了一个向量查询 FunctionTool.from_defaults。因此，我们将 vector_query 函数传递到向量查询工具中，这使得我们可以将其与语言模型一起使用。

因此，让我们借助 gpt-3.5-turbo 调用此工具。我们会发现 **LLM 能够推断字符串以及元数据过滤器**。我们在向量查询工具上进行了 predict_and_call，并提示了同样的问题：如第二页所述的 MetaGPT 的高水平结果。在这里我们看到 LLM 能够制定正确的查询（MetaGPT 的高水平结果），以及指定页码（取值为 "2"）。结果是我们得到了正确的答案。

In [16]:
from typing import List, Tuple
from llama_index.core.vector_stores import FilterCondition


def vector_query(query: str, page_number: int) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (int): Filter by set of pages. Leave BLANK if we want to perform a vector search over all pages. Otherwise, filter by the set of specified pages.

    """
    print(page_number, type(page_number))
    metadata_dicts = [{"key": "page_label", "value": str(page_number)}]
    print(metadata_dicts)

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )
    response = query_engine.query(query)

    return response


vector_query_tool = FunctionTool.from_defaults(name="vector_tool", fn=vector_query)

In [17]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool],
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 2, "query": "high-level results of MetaGPT"}
2 <class 'int'>
[{'key': 'page_label', 'value': '2'}]
=== Function Output ===
MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with 85.9% and 87.7% in Pass@1. It stands out in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, MetaGPT achieves a 100% task completion rate, demonstrating robustness and efficiency in terms of time and token costs.


In [19]:
# 中文
def vector_query(query: str, page_number: int) -> str:
    """对索引进行向量搜索。

    query (str)：要进行嵌入的字符串查询。
    page_numbers (int)：按页面集过滤。如果要在所有页面上进行向量搜索，则留空。否则，按指定的页面集过滤。

    """
    print(page_number, type(page_number))
    metadata_dicts = [{"key": "page_label", "value": str(page_number)}]
    print(metadata_dicts)

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )
    response = query_engine.query(query)

    return response


vector_query_tool = FunctionTool.from_defaults(name="vector_tool", fn=vector_query)

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], "第 2 页中描述的 MetaGPT 的高水平结果是什么？请用中文回答。", verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 2, "query": "MetaGPT \u7684\u9ad8\u6c34\u5e73\u7ed3\u679c"}
2 <class 'int'>
[{'key': 'page_label', 'value': '2'}]
=== Function Output ===
MetaGPT 在代码生成基准测试中取得了新的技术水平，Pass@1 分别为 85.9% 和 87.7%，在处理更高级别的软件复杂性和提供广泛功能方面表现出色。在实验评估中，MetaGPT 实现了 100% 的任务完成率，展示了设计的稳健性和效率。


我们可以快速验证源节点，可以看到源节点的页面标签，有一个返回的源节点，它的页面标签是 "2"。

In [20]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


## 三、与向量检索工具结合起来

最后，我们可以从 2. 路由查询引擎 Router Query Engine 教程中的 router 示例中引入摘要工具，并将其与向量检索工具有机结合起来，创建这个综合的工具选择系统。所以，下面这段代码只是在相同的节点集上设置了一个摘要索引，并用一个摘要工具将其包装起来。


In [21]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("Useful if you want to get a summary of MetaGPT."),
)

现在我们再次尝试工具调用。这样一来，**LLM 要做的任务略有提高难度**，它不仅要实际选择正确的工具，还要推断函数参数。我们询问论文第 8 页上 MetaGPT 与 ChatDev 的比较情况。我们看到它实际上还是调用了一个向量工具，页码等于 "8"，而且它能够给出正确的答案。


In [22]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 8, "query": "MetaGPT comparisons with ChatDev"}
8 <class 'int'>
[{'key': 'page_label', 'value': '8'}]
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various metrics. For example, MetaGPT achieves a higher score in executability, takes less time for software generation, uses more tokens but requires fewer tokens to generate one line of code compared to ChatDev. Additionally, MetaGPT demonstrates better performance in code statistics and human revision cost when compared to ChatDev.


我们还可以通过打印出来源节点来验证这一点。

In [23]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


In [27]:
# 中文
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("如果您想了解 MetaGPT 的概要，这个工具会很有用。"),
)

response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "第 8 页描述的 MetaGPT 与 ChatDev 的对比分析结果是什么？必须用中文回答。",
    verbose=True,
)

for n in response.source_nodes:
    print(n.metadata)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 8, "query": "MetaGPT \u4e0e ChatDev \u7684\u5bf9\u6bd4\u5206\u6790\u7ed3\u679c"}
8 <class 'int'>
[{'key': 'page_label', 'value': '8'}]
=== Function Output ===
MetaGPT 在软件开发数据集中几乎在所有指标上均优于 ChatDev。例如，在可执行性方面，MetaGPT 获得了3.75的分数，接近4（完美）。此外，MetaGPT 生成一行代码所需的时间更短（503秒），明显少于 ChatDev。考虑到代码统计和人工修订成本，MetaGPT 也明显优于 ChatDev。虽然 MetaGPT 需要更多的标记（24,613或31,255比19,292多），但只需要126.5/124.3个标记来生成一行代码。相比之下，ChatDev 使用了248.9个标记。这些结果突显了在多个代理之间的协作中 SOP 的好处。
{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


最后，我们可以问一个问题，即**这篇论文的摘要是什么**，以显示 LLM 在必要时仍然能够选择摘要工具。我们看到它给出了正确的回应。

In [36]:
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("Useful if you want to get a summary of MetaGPT."),
)

response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What is a summary of the MetaGPT paper?",
    verbose=True,
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Summary of the MetaGPT paper"}
=== Function Output ===
The MetaGPT paper introduces a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to enhance multi-agent systems based on Large Language Models (LLMs). It focuses on role specialization, structured communication interfaces, and executable feedback mechanisms to improve code generation quality during runtime. The framework involves agents like Product Managers, Architects, Engineers, and QA Engineers, each contributing specialized outputs to efficiently complete software development tasks. Additionally, it introduces the concept of AgentStore for creating and developing agents within the framework. The paper discusses the transformation of abstract requirements into detailed software designs, emphasizing the importance of structured messaging and feedback mechanisms. It addresses challenges such as reducing code hallucination

In [45]:
# 中文
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("如果您想了解 MetaGPT 的概要，这个工具会很有用。"),
)

# 发现使用 LlamaIndex 框架，即使是中文 Prompt，仍可能出现不遵循指令的情况。多次调试，输出结果仍是英文。
# 因此，感兴趣的读者可以换一篇中文的论文 PDF 来做实验！
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], "用中文输出 MetaGPT 论文的概要。", verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "MetaGPT"}
=== Function Output ===
MetaGPT is a meta-programming framework that enhances problem-solving capabilities in multi-agent systems based on Large Language Models (LLMs). It incorporates role specialization, structured communication interfaces, and efficient sharing mechanisms to facilitate collaboration among agents. MetaGPT also utilizes an executable feedback mechanism to improve code generation quality during runtime. In experiments, MetaGPT has shown state-of-the-art performance on various benchmarks, outperforming other frameworks in tasks like code generation and software development. Additionally, MetaGPT is a framework that utilizes natural language programming to generate software applications based on provided prompts and requirements. It simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and standard

## 三、本章小结
***

🧍🐒🤖 人与大多数动物的根本区别在于人能制造和使用工具。同样地，智能体也能利用各种外部工具来实现功能扩展和效率提升。

在本教程中，我们展示了如何使用 LLM 来选择要执行的函数，并推断要传递给函数的参数。通过工具调用，LLM 不仅能够选择合适的工具，还能决定要给工具什么参数。这使得用户能够提出更多问题，并通过工具调用获得比标准 RAG 技术更精确的结果。

教程中还展示了如何在向量搜索之上定义一个稍微复杂一点的智能体层。LLM 可以选择向量搜索，并推断元数据过滤器，以返回更精确的搜索结果。最后，我们展示了如何将整体检索工具封装成一个函数，使 LLM 能够推断用户查询的元数据过滤器。而且通过 LlamaIndex 索引抽象，我们还可以定义各种元数据过滤器，如章节 IDs、页码、页眉、页脚等。

✨️🎉🚀 那么，**3. 工具调用 Tool Calling** 就到这里啦~ ✨️🎉🚀 

🤖 下一章 **4. Building an Agent Reasoning Loop**，我们将向您展示如何构建智能体推理闭环！🤖